### Project 1

We need to design an implement a class that will be used to represent bank accounts.

We want the following functionality and characteristics:
- accounts are uniquely identified by an **account number** (assume it will just be passed in the initializer)
- account holders have a **first** and **last** name
- accounts have an associated **preferred time zone offset** (e.g. -7 for MST)
- **balances** need to be zero or higher, and should not be directly settable.
- but, **deposits** and **withdrawals** can be made (given sufficient funds)
    - if a withdrawal is attempted that would result in nagative funds, the transaction should be declined.
- a **monthly interest rate** exists and is applicable to all accounts **uniformly**. There should be a method that can be called to calculate the interest on the current balance using the current interest rate, and **add it** to the balance.
- each deposit and withdrawal must generate a **confirmation number** composed of:
    - the transaction type: `D` for deposit, and `W` for withdrawal, `I` for interest deposit, and `X` for declined (in which case the balance remains unaffected)
    - the account number
    - the time the transaction was made, using UTC
    - an incrementing number (that increments across all accounts and transactions)
    - for (extreme!) simplicity assume that the transaction id starts at zero (or whatever number you choose) whenever the program starts
    - the confirmation number should be returned from any of the transaction methods (deposit, withdraw, etc)
- create a **method** that, given a confirmation number, returns:
    - the account number, transaction code (D, W, etc), datetime (UTC format), date time (in whatever timezone is specified in te argument, but more human readable), the transaction ID
    - make it so it is a nicely structured object (so can use dotted notation to access these three attributes)
    - I purposefully made it so the desired timezone is passed as an argument. Can you figure out why? (hint: does this method require any information from any instance?)

For example, we may have an account with:
- account number `140568` 
- preferred time zone offset of -7 (MST) 
- an existing balance of `100.00`

Suppose the last transaction ID in the system was `123`, and a deposit is made for `50.00` on `2019-03-15T14:59:00` (UTC) on that account (or `2019-03-15T07:59:00` in account's preferred time zone offset)

The new balance should reflect `150.00` and the confirmation number returned should look something like this:

```D-140568-20190315145900-124```

We also want a method that given the confirmation number returns an object with attributes:
- `result.account_number` --> `140568`
- `result.transaction_code` --> `D`
- `result.transaction_id` --> `124`
- `result.time` --> `2019-03-15 07:59:00 (MST)`
- `result.time_utc` --> `2019-03-15T14:59:00`

Furthermore, if current interest rate is `0.5%`, and the account's balance is `1000.00`, then the result of calling the `deposit_interest` (or whatever name you choose) method, should result in a new transaction and a new balance of `1050.00`. Calling this method should also return a confirmation number.

For simplicty, just use floats, but be aware that for these types of situations you'll probably want to use `Decimal` objects instead of floats.

There are going to be many ways to design something like this, especially since I have not nailed down all the specific requirements, so you'll have to fill the gaps yourself and decide what other things you may want to implement (like is the account number going to be a mutable property, or "read-only" and so on).

See how many different ideas you can use from what we covered in the last section. 

My approach will end up creating two classes: a `TimeZone` class used to store the time zone name and offset definition (in hours and minutes), and a main class called `Account` that will have the following "public" interface:
- initializer with account number, first name, last name, optional preferred time zone, starting balance (defaults to 0)
- a first name property (read/write)
- a last name property (read/write)
- a full name property (computed, read-only)
- a balance property (read-only)
- an interest rate property (class level property)
- deposit, withdraw, pay_interest methods
- parse confirmation code

Class will have additional state and methods, but those will be used for implementation.

You should also remember to test your code! In the solutions I will introduce you to Python's `unittest` package. Even if you skip this project, at least review that video and/or notebook if you are unfamiliar with `unittest`.

In [14]:
# timezone class
import numbers
from datetime import timedelta

class TimeZone:
    def __init__(self, name, offset_hours, offset_minutes):
        if name is None or len(str(name).strip()) == 0:
            raise ValueError('Timezone name cannot be empty.')
        
        self._name = str(name).strip()
        
        if not isinstance(offset_hours, numbers.Number):
            raise ValueError("Hour offset must be an integer")
        
        if not isinstance(offset_minutes, numbers.Number):
            raise ValueError('Minute offset must be an integer')
        
        if offset_minutes > 59 or offset_minutes < -59:
            raise ValueError('Minutes offset must be between -59 and 59 (inclusive).')
        
        offset = timedelta(hours = offset_hours, minutes = offset_minutes)
        if offset < timedelta(hours = -12, minutes = 0) or offset > timedelta(hours = 14, minutes =0 ):
            raise ValueError('Offset must be between -12:00 and +14:00')
        
        self._offset_hours = offset_hours
        self._offset_minutes = offset_minutes
        self._offset = offset
        
    @property
    def offset(self):
        return self._offset

    @property
    def name(self):
        return self._name

    def __eq__(self,other):
        return (isinstance(other,TimeZone) and 
                self.name == other.name and
                self._offset_minutes == other._offset_minutes and
                self._offset_hours == other._offset_hours)

    def __repr__(self):
        return (f"TimeZone(name= '{self.name}',"
                f"offset_hours = {self._offset_hours},"
                f"offset_minutes = {self._offset_minutes})")


In [15]:
tz1 = TimeZone('ABC', -2,-15)

In [17]:
tz1.__dict__
tz1.name


'ABC'

In [18]:
from datetime import datetime
dt = datetime.utcnow()
dt

datetime.datetime(2024, 4, 9, 16, 14, 15, 457363)

In [19]:
print(dt)

2024-04-09 16:14:15.457363


In [20]:
print(dt+tz1.offset)

2024-04-09 13:59:15.457363


In [22]:
try:
    tz = TimeZone(" ",0,0)
except ValueError as ex:
    print(ex)

Timezone name cannot be empty.


In [23]:
class TransactionID:
    def __init__(self,start_id):
        self._start_id = start_id
    
    def next(self):
        self._start_id +=1
        return self._start_id

In [41]:
def transaction_ids(start_id):
    while True:
        start_id+=1
        yield start_id
        
class Account:
    interest_rate = 0.005
    
    transaction_counter = transaction_ids(100)
    
    def make_transaction(self):
        new_trans_id = next(Account.transaction_counter)
        return new_trans_id
#     def __init__(self,accountID,firstName,lastName,preferred_time_zone, balance = 0):
#         self._accountID = accountID
#         self._lastName = lastName
#         self._firstName = firstName
#         self.balance = balance
    
#     @property
#     def firstName(self):
#         return self._firstName
    
#     @firstName.setter
#     def firstName(self,value):
#         self._firstName = value

#     @property
#     def lastName(self):
#         return self._lastName
    
#     @lastName.setter
#     def firstName(self,value):
#         self._lastName = value
    
#     @property
#     def fullName(self):
#         return f"{self._firstName} {self._lastName}"
    
    

In [42]:
a1 = Account()
a2 = Account()


In [43]:
print(a1.make_transaction())
print(a2.make_transaction())
print(a1.make_transaction())

101
102
103


In [34]:
t = transaction_ids(100)

In [35]:
t

<generator object transaction_ids at 0x7fa0302fb120>

In [36]:
next(t)

101

In [37]:
next(t)

102

In [143]:
import itertools

class Account:
    _interest_rate = 0.5   # percent/ class property
    
    _transaction_codes = {
        'deposit':'D',
        'withdraw':'W',
        'interest':'I',
        'rejected':'X'
    }
    
    transaction_counter = itertools.count(100)
    
    def make_transaction(self):
        new_trans_id = next(Account.transaction_counter)
        return new_trans_id
    
    def __init__(self,account_number, first_name, last_name,timezone = None, initial_balance = 0):
        self._account_number = account_number
        self.first_name = first_name
        self.last_name = last_name
        
        if timezone is None:
            timezone = TimeZone('UTC',0,0)
        self._timezone = timezone
        self._balance = float(initial_balance)
        
    @property
    def account_number(self):
        return self._account_number
    
    @property
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self,value):

        self._first_name = Account.validate_name(value, 'First Name')
    
    @property
    def last_name(self):
        return self._last_name
    
    @last_name.setter
    def last_name(self,value):
        self._last_name = Account.validate_name(value, 'Laste Name')  
    
    @property
    def timezone(self):
        return self._timezone
    
    @timezone.setter
    def timezone(self,value):
        if not isinstance(value,TimeZone):
            raise ValueError('Time Zone must be a valid TimeZone object')
        self._timezone = value
    
    @property
    def balance(self):
        return self._balance
    @staticmethod
    def validate_name(value,field_title):
        if not value or len(value.strip()) == 0:
            raise ValueError(f'{field_title} cannot be empty')
        return str(value).strip()
    
    @classmethod
    def get_interest_rate(cls):
        return cls._interest_rate
    
    @classmethod
    def set_interest_rate(cls, value):
        if not isinstance(value, numbers.Integral):
            raise ValueError(f'interest rate must be a number')
        
        if value < 0:
            raise ValueError(f'interest rate must be postive')
        
        cls._interest_rate = value

    from datetime import datetime

    def generate_confirmation_code(self,transaction_code):
        dt_str = datetime.utcnow().strftime('%Y%m%d%H%M%S')
        return f'{transaction_code}-{self.account_number}-{dt_str}-{next(Account.transaction_counter)}'
    
    def make_transaction(self):
        return self.generate_confirmation_code('dummy')

    from collections import namedtuple

    Confirmation = namedtuple('Confirmation','account_number confirmation_code transaction_id time_utc time')

    @staticmethod
    def parse_confirmation_code(confirmation_code,preferred_time_zone = None):
        # dummy
        parts = confirmation_code.split('-')
        if len(parts) != 4:
            raise ValueError('Invalid confirmation code')
        
        transaction_code, account_number, raw_dt_utc, transaction_id = parts
        try:
            dt_utc = datetime.strptime(raw_dt_utc,'%Y%m%d%H%M%S')
        except ValueError as ex:
            raise ValueError('Invalid transaction datetime.') from ex
        
        if preferred_time_zone is None:
            preferred_time_zone = TimeZone('UTC',0,0)
        if not isinstance(preferred_time_zone, TimeZone):
            raise ValueError('Invalid TimeZone specificed')
        
        dt_preferred = dt_utc + preferred_time_zone.offset
        dt_preferred_str = f"{dt_preferred.strftime('%Y-%m-%d %H:%M%S')} ({preferred_time_zone.name})"
        
        return Confirmation(account_number, transaction_code, transaction_id, dt_utc.isoformat(), dt_preferred_str)


In [144]:
a = Account('A100','Eric','Idle')
conf_code = a.make_transaction()
print(conf_code)

dummy-A100-20240409215105-100


In [145]:
Account.parse_confirmation_code(conf_code)

NameError: name 'Confirmation' is not defined

In [134]:
try:
    a = Account('234','John',None)
except ValueError as ex:
    print(ex)

Laste Name cannot be empty


In [127]:
try:
    a = Account('234','John','Cleese','-7:00')
except ValueError as ex:
    print(ex)

In [130]:
a = Account('A100','John','Cleese')
a.make_transaction()
a2 = Account('123','dumsdfe','sdf')
a2.make_transaction()

'dummy-123-20240409213504-104'

In [131]:
a = Account('234','John','Cleese',initial_balance = 100)
a.balance
try:
    a.balance = 200
except AttributeError as ex:
    print(ex)

can't set attribute


In [115]:
a1 = Account('234','John','Cleese')
a2 = Account('2345','John','Cleese')
print(a1.get_interest_rate())
print(a2.get_interest_rate())
a1.set_interest_rate(10) 
print(a1.get_interest_rate())
print(a2.get_interest_rate())
a2.set_interest_rate(100)
print(a1.get_interest_rate())
print(a2.get_interest_rate())

100
100
10
10
100
100


In [81]:
Account.interest_rate

0.5

In [88]:
# reflect through the class
Account.interest_rate = 10
a1.get_interest_rate, a2.get_interest_rate

(<bound method Account.get_interest_rate of <class '__main__.Account'>>,
 <bound method Account.get_interest_rate of <class '__main__.Account'>>)

In [92]:
try:
    Account.set_interest_rate(1+1j)
except ValueError as ex:
    print(ex)

interest rate must be a number


In [83]:
a1.interest_rate = 100
a2.interest_rate

10

In [84]:
a1.__dict__

{'_account_number': '234',
 '_first_name': 'John',
 '_last_name': 'Cleese',
 '_timezone': TimeZone(name= 'UTC',offset_hours = 0,offset_minutes = 0),
 '_balance': 0.0,
 'interest_rate': 100}

In [46]:
a1 = Account()
a2 = Account()
print(a1.make_transaction())
print(a2.make_transaction())
print(a1.make_transaction())

100
101
102


In [116]:
#help(itertools)

In [117]:
from datetime import datetime

def generate_confirmation_code(account_number, transaction_id, transaction_code):
    dt_str = datetime.utcnow().strftime('%Y%m%d%H%M%S')
    return f'{transaction_code}-{account_number}-{dt_str}-{transaction_id}'
    

In [118]:
generate_confirmation_code(123,1000,'X')

'X-123-20240409213146-1000'

In [ ]:
from collections import namedtuple

Confirmation = namedtuple('Confirmation','account_number confirmation_code transaction_id time_utc time')